<img src="./pictures/logo-insa.png" style="float:right; max-width: 60px; display: inline" alt="INSA" /></a>

# Propeller selection
*Written by Marc Budinger, INSA Toulouse, France*

## Design graph 

The following diagram represents the design graph of the propeller’s selection. The max thrust is assumed to be known here.


![DesignGraph](pictures/PropellerDesignGraph.png)

> **Questions:**
* Give the main sizing problems you are able to detect.
* Propose one or multiple solutions (which can request equation manipulation, addition of design variables, addition of constraints) 
* Orientate the arrows
* Give equations order, inputs/outputs at each step of this part of sizing procedure



### Sizing code and optimization

> Exercice: propose a sizing code for the selection of a propeller.


In [1]:
from utils.model_standard import CoreModel
from utils.model_serializer import ModelSerializer

In [2]:
class PropellerModel(CoreModel):
    """
    Propeller model class.
    ----------
    """

    def __init__(self, **kwargs):
        super(PropellerModel, self).__init__(**kwargs)
        self.initialization()
        self.execute()
        self._update()

    def initialization(self):
        
        # Input variables
        inputs = {'k_ND': 1.0, 'beta': 0.5, 'Tpro_takeoff': 15.0, 'Tpro_hover': 5.0}
        self.set_inputs(inputs)
        
        # Input parameters
        
        # Propeller selection
        # ---
        rho_air=1.18 # [kg/m^3] air density 
        NDmax=105000/60*.0254 # [Hz.m] max speed limit (N.D max)
        Dpro_ref=11*.0254 # [m] diameter
        Mpro_ref=0.53*0.0283 # [kg] mass
        
        inputs = {'rho_air': rho_air, 'NDmax': NDmax, 'Dpro_ref': Dpro_ref, 'Mpro_ref': Mpro_ref}
        self.set_inputs(inputs)
               
        # Declare outputs
        outputs = ['C_t', 'C_p', 'Dpro', 'n_pro_takeoff', 'Wpro_takeoff', 'Mpro', 'Ppro_takeoff', 'Qpro_takeoff', 'P_el_hover', 'n_pro_hover', 'Wpro_hover', 'Ppro_hover', 'Qpro_hover']   
        self.declare_outputs(outputs)

    def execute(self):
        
        # Get input values
        k_ND, beta, Tpro_takeoff, Tpro_hover = self.get_values(['k_ND', 'beta', 'Tpro_takeoff', 'Tpro_hover'])
        
        rho_air, NDmax, Dpro_ref, Mpro_ref = self.get_values(['rho_air', 'NDmax', 'Dpro_ref', 'Mpro_ref'])

        
        #######
        
        C_t = 4.27e-02 + 1.44e-01 * beta 
        
        #security coefficient
        ND = k_ND*NDmax   #on suppose donc k_ND compris entre 0 et 1
        
        #C_t = Tpro_takeoff/(rho_air*NDmax**2*Dpro_ref**2)
        Dpro = (Tpro_takeoff/(C_t*rho_air*ND**2))**(1/2)  #diamètre de l'hélice permettant de générer l'effort requis
        n_pro_takeoff = ND/Dpro
        
        C_p = -1.48e-03 + 9.72e-02 * beta
        
        Ppro_takeoff = C_p*rho_air*n_pro_takeoff**2*Dpro**4 #puissance développée
        
        Wpro_takeoff = 2*3.1416*n_pro_takeoff  #vitesse de rotation
        Qpro_takeoff= Ppro_takeoff/(Wpro_takeoff) #couple 
        
        Mpro = Dpro**2  #loi de masse
        
        #caractéristiques du propeller en phase hover
        Dpro_hover = (Tpro_hover/(C_t*rho_air*ND**2))**(1/2)
        n_pro_hover = ND/Dpro_hover  
        Ppro_hover = C_p*rho_air*n_pro_hover**2*Dpro_hover**4
        Wpro_hover = 2*3.1416*n_pro_hover   
        Qpro_hover= Ppro_hover/(Wpro_hover)
        
        ########
        
        outputs = {'C_t': C_t, 'C_p': C_p, 'Dpro': Dpro, 'n_pro_takeoff': n_pro_takeoff, 'Wpro_takeoff': Wpro_takeoff, 'Mpro': Mpro}
        self.set_outputs(outputs)
        
        outputs = {'Ppro_takeoff': Ppro_takeoff, 'Qpro_takeoff': Qpro_takeoff}
        self.set_outputs(outputs)
        
        outputs = {'n_pro_hover': n_pro_hover, 'Wpro_hover': Wpro_hover, 'Ppro_hover': Ppro_hover, 'Qpro_hover': Qpro_hover}
        self.set_outputs(outputs)
        
    def __str__(self):
        
        s =(("* Propellers informations: \n") +
            ("** Global: \n") + 
            ("    NxD takeoff = %.0f RPMxInch"%(self.get_values(['n_pro_takeoff'])*60*self.get_values(['Dpro'])/.0254) + "\n") +
            ("    Diameter Dpro = %.2g m or %.2f in"%((self.get_values(['Dpro']), self.get_values(['Dpro'])/.0254)) + "\n") +
            ("    Pitch  = %.2g m or %.2f in"%(self.get_values(['beta'])*self.get_values(['Dpro']), self.get_values(['beta'])*self.get_values(['Dpro'])/.0254) + "\n") +
            ("    Propeller mass (1x): %.3f kg"%(self.get_values(['Mpro'])) + "\n") +
            ("** Aerodynamics: \n") +
            ("    Power coefficient C_p: - for statics: %.4f "%(self.get_values(['C_p'])) + "\n") +
            ("    Thrust coefficient C_t: - for statics: %.4f "%(self.get_values(['C_t']))+ "\n") +
            ("** Rotational speeds: \n") +
            ("    - for hover: %.0f RPM "%(self.get_values(['Wpro_hover'])*60/2/3.1416) + "\n") +
            ("    - for vertical acceleration: %.0f RPM "%(self.get_values(['Wpro_takeoff'])*60/2/3.14) + "\n")
           )
        return s
        

In [3]:
propeller_model = PropellerModel()

print(propeller_model)

inputs = {'Tpro_takeoff': 25.0}

propeller_model.evaluate(inputs, [])

print(propeller_model)

* Propellers informations: 
** Global: 
    NxD takeoff = 105000 RPMxInch
    Diameter Dpro = 0.24 m or 9.32 in
    Pitch  = 0.12 m or 4.66 in
    Propeller mass (1x): 0.056 kg
** Aerodynamics: 
    Power coefficient C_p: - for statics: 0.0471 
    Thrust coefficient C_t: - for statics: 0.1147 
** Rotational speeds: 
    - for hover: 19504 RPM 
    - for vertical acceleration: 11267 RPM 

* Propellers informations: 
** Global: 
    NxD takeoff = 105000 RPMxInch
    Diameter Dpro = 0.31 m or 12.04 in
    Pitch  = 0.15 m or 6.02 in
    Propeller mass (1x): 0.093 kg
** Aerodynamics: 
    Power coefficient C_p: - for statics: 0.0471 
    Thrust coefficient C_t: - for statics: 0.1147 
** Rotational speeds: 
    - for hover: 19504 RPM 
    - for vertical acceleration: 8727 RPM 



In [4]:
propeller_model = PropellerModel()

print(propeller_model)

ms = ModelSerializer()
path = './models_student/'
file_name = 'propeller_model'
ms.save_model(propeller_model, path + file_name)
    

* Propellers informations: 
** Global: 
    NxD takeoff = 105000 RPMxInch
    Diameter Dpro = 0.24 m or 9.32 in
    Pitch  = 0.12 m or 4.66 in
    Propeller mass (1x): 0.056 kg
** Aerodynamics: 
    Power coefficient C_p: - for statics: 0.0471 
    Thrust coefficient C_t: - for statics: 0.1147 
** Rotational speeds: 
    - for hover: 19504 RPM 
    - for vertical acceleration: 11267 RPM 



In [5]:
new_model = ms.load_model(path + file_name)

### Commentaires sur le propeller : 
- fonction : créer de la poussée 
- imperfections : rendement propulsif (discload) qu'il faut garder suffisamment bon 
- quand la vitesse augmente avec le Mach qui tend vers 1, le couple et la puissance deviennent trop importants, on a donc une limite en vitesse de rotation
- scénarios : pour le take-off, les performances peuvent être dégradées et c'est la phase dimensionnante ; pour le vol stationnaire, on veut surtout avoir un bon rendement et une autonomie importante 
